In [ ]:
import logging
import os
import urllib3

logging.captureWarnings(True)
http = urllib3.PoolManager()

repo_base = "https://raw.githubusercontent.com/earwole1/covid/master/"
files= ["covid.py", "population.py"]

for file in files:
  url = os.path.join(repo_base, file)
  r = http.request("GET", url)
  out_name = os.path.basename(url)
  with open(out_name, 'wb') as fp:
    fp.write(r.data)
    print(f"Copied {out_name}")


In [ ]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from covid import Covid
c = Covid()

In [ ]:
c.pull_and_save("https://github.com/nytimes/covid-19-data/raw/master/rolling-averages/us-states.csv", "rolling.csv")

In [ ]:
r = pd.read_csv("rolling.csv")
r.date = pd.to_datetime(r.date, format="%Y-%m-%d")

In [ ]:
def plot_state(state):
  st = r[r.state == state]
  st.sort_values("date", ascending=True, inplace=True)
  fig, ax = plt.subplots()
  l1 = ax.plot(st.date, st.cases_avg, 'r-', label='Cases/Day')
  ax.grid()
  fig.set_figwidth(15)
  fig.set_figheight(10)
  ax.set_ylabel('Cases/Day')
  ax2 = ax.twinx()
  l2  = ax2.plot(st.date, st.deaths_avg, 'b--', label='Deaths/Day')
  ax2.set_ylabel('Deaths/Day')
  _ = ax.legend(loc='upper left')
  _ = ax2.legend(loc='lower right')
  ax.set_title(f"Per Day stats for {state}")

  fig, ax = plt.subplots()
  l1 = ax.plot(st.date, np.cumsum(st.cases_avg), 'r-', label='Total Cases')
  ax.grid()
  fig.set_figwidth(15)
  fig.set_figheight(10)
  ax.set_ylabel('Cases')
  ax2 = ax.twinx()
  l2  = ax2.plot(st.date, np.cumsum(st.deaths_avg), 'b--', label='Total Deaths')
  ax2.set_ylabel('Deaths')
  _ = ax.legend(loc='upper left')
  _ = ax2.legend(loc='lower right')
  ax.set_title(f"Cummukative stats for {state}")

  dr = np.cumsum(st.deaths_avg) / np.cumsum(st.cases_avg)
  dr[np.isinf(dr)] = 0
  dr *= 100.0 
  fig, ax = plt.subplots()
  _ = ax.plot(st.date, dr, 'k-', label='Death Rate')
  _ = ax.legend()
  fig.set_figwidth(15)
  fig.set_figheight(10)
  ax.grid()
  return st


In [ ]:
md = plot_state("Maryland")

In [ ]:
oh = plot_state("Ohio")

In [ ]:
fl = plot_state("Florida")

In [ ]:
states = r.state.unique()
tot = {}
for s in states:
  q = r[r.state == s]
  q.sort_values("date", ascending=True, inplace=True)
  q = q[-350:]
  _cases = q.cases_avg.sum()
  _deaths = q.deaths_avg.sum()
  if _cases == 0:
    dr = 0
  else:
    dr = _deaths / _cases * 100

  tot[s] = {"cases": q.cases_avg.sum(), "deaths": q.deaths_avg.sum(), "dr": dr}

In [ ]:
df = pd.DataFrame(tot).T
df.sort_values("dr", ascending=False, inplace=True)


In [ ]:
df.head(20)